In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
import sqlite3 as sql


To solve the data consistency issue, decompose the original dataframe into multiple tables.




- Define a master dataframe called transactions, which records the vendor number, store number, product (item number), transaction date, bottles sold, volume (either in gallons or in liters), sales amount.



In [2]:
tr = pd.read_csv('Iowa_Liquor_Sales.csv')

c:\users\amd-pc\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
tr.describe()

,Store Number,County Number,Category,Vendor Number,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
count,2.096600e+07,2.080920e+07,2.094902e+07,2.096599e+07,2.096600e+07,2.096600e+07,2.096599e+07,2.096599e+07,2.096600e+07,2.096599e+07,2.096600e+07,2.096600e+07
mean,3.708108e+03,5.729485e+01,1.047362e+06,2.611853e+02,1.226488e+01,9.066495e+02,1.000865e+01,1.502688e+01,1.039690e+01,1.362798e+02,9.135539e+00,2.411741e+00
std,1.041117e+03,2.728383e+01,7.067343e+04,1.403348e+02,7.686060e+00,6.568687e+02,1.069581e+01,1.604326e+01,2.809012e+01,4.680959e+02,3.379455e+01,8.927640e+00
min,2.106000e+03,1.000000e+00,1.012200e+05,1.000000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.614000e+03,3.100000e+01,1.012200e+06,1.150000e+02,6.000000e+00,7.500000e+02,5.500000e+00,8.250000e+00,3.000000e+00,3.196000e+01,1.500000e+00,4.000000e-01
50%,3.782000e+03,6.200000e+01,1.031200e+06,2.600000e+02,1.200000e+01,7.500000e+02,8.250000e+00,1.238000e+01,6.000000e+00,7.200000e+01,4.800000e+00,1.270000e+00
75%,4.551000e+03,7.700000e+01,1.062400e+06,3.800000e+02,1.200000e+01,1.000000e+03,1.224000e+01,1.838000e+01,1.200000e+01,1.417200e+02,1.050000e+01,2.770000e+00
max,9.946000e+03,9.900000e+01,1.901200e+06,9.870000e+02,3.360000e+02,3.780000e+05,7.680000e+03,1.152000e+04,1.500000e+04,2.795573e+05,1.500000e+04,3.962580e+03


In [4]:
tr.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S31296100034,03/16/2016,3762,Wine and Spirits Gallery,7690 HICKMAN RD,WINDSOR HEIGHTS,50322,NaN,77.0,Polk,...,80457,Ryan's Cream Liqueur,12,1000,7.23,10.85,1,10.85,1.0,0.26
1,INV-20599500022,07/15/2019,4320,Fareway Stores #470 / Perry,NaN,NaN,NaN,NaN,NaN,NaN,...,10008,Scoresby Rare Scotch,6,1750,10.50,15.75,6,94.50,10.5,2.77
2,INV-28403900139,07/01/2020,2643,Hy-Vee Wine and Spirits / Waterloo,2126 Kimball Ave,Waterloo,50701,NaN,7.0,BLACK HAWK,...,15628,Jameson,6,1750,32.48,48.72,2,97.44,3.5,0.92
3,INV-20560900100,07/11/2019,3456,Quick Shop / Clear Lake,NaN,NaN,NaN,NaN,NaN,NaN,...,10009,Scoresby Rare Scotch,12,1000,6.74,10.11,2,20.22,2.0,0.52
4,INV-20784900043,07/23/2019,5151,IDA Liquor,"500, Hwy 175",Ida Grove,51445,NaN,47.0,IDA,...,100107,Captain Morgan OSR Glass w/50ml Apple Smash & ...,6,1850,18.00,27.00,6,162.00,11.1,2.93


In [5]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20965997 entries, 0 to 20965996
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Invoice/Item Number    object 
 1   Date                   object 
 2   Store Number           int64  
 3   Store Name             object 
 4   Address                object 
 5   City                   object 
 6   Zip Code               object 
 7   Store Location         object 
 8   County Number          float64
 9   County                 object 
 10  Category               float64
 11  Category Name          object 
 12  Vendor Number          float64
 13  Vendor Name            object 
 14  Item Number            object 
 15  Item Description       object 
 16  Pack                   int64  
 17  Bottle Volume (ml)     int64  
 18  State Bottle Cost      float64
 19  State Bottle Retail    float64
 20  Bottles Sold           int64  
 21  Sale (Dollars)         float64
 22  Volume Sold (Lit

In [6]:
np.sum(tr.isna())

Invoice/Item Number            0
Date                           0
Store Number                   0
Store Name                     0
Address                    79992
City                       79991
Zip Code                   80036
Store Location           2010426
County Number             156796
County                    156794
Category                   16974
Category Name              25040
Vendor Number                  8
Vendor Name                    6
Item Number                    0
Item Description               0
Pack                           0
Bottle Volume (ml)             0
State Bottle Cost             10
State Bottle Retail           10
Bottles Sold                   0
Sale (Dollars)                10
Volume Sold (Liters)           0
Volume Sold (Gallons)          0
dtype: int64

In [7]:
tr.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S31296100034,03/16/2016,3762,Wine and Spirits Gallery,7690 HICKMAN RD,WINDSOR HEIGHTS,50322,NaN,77.0,Polk,...,80457,Ryan's Cream Liqueur,12,1000,7.23,10.85,1,10.85,1.0,0.26
1,INV-20599500022,07/15/2019,4320,Fareway Stores #470 / Perry,NaN,NaN,NaN,NaN,NaN,NaN,...,10008,Scoresby Rare Scotch,6,1750,10.50,15.75,6,94.50,10.5,2.77
2,INV-28403900139,07/01/2020,2643,Hy-Vee Wine and Spirits / Waterloo,2126 Kimball Ave,Waterloo,50701,NaN,7.0,BLACK HAWK,...,15628,Jameson,6,1750,32.48,48.72,2,97.44,3.5,0.92
3,INV-20560900100,07/11/2019,3456,Quick Shop / Clear Lake,NaN,NaN,NaN,NaN,NaN,NaN,...,10009,Scoresby Rare Scotch,12,1000,6.74,10.11,2,20.22,2.0,0.52
4,INV-20784900043,07/23/2019,5151,IDA Liquor,"500, Hwy 175",Ida Grove,51445,NaN,47.0,IDA,...,100107,Captain Morgan OSR Glass w/50ml Apple Smash & ...,6,1850,18.00,27.00,6,162.00,11.1,2.93


Reduce dataset. Drop Item Numbers that have < 100 transactions.

In [8]:
item_list = tr['Item Number'].value_counts()[tr['Item Number'].value_counts() >= 100].index
tr = tr.loc[tr['Item Number'].isin(item_list)]

In [9]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20809628 entries, 0 to 20965996
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Invoice/Item Number    object 
 1   Date                   object 
 2   Store Number           int64  
 3   Store Name             object 
 4   Address                object 
 5   City                   object 
 6   Zip Code               object 
 7   Store Location         object 
 8   County Number          float64
 9   County                 object 
 10  Category               float64
 11  Category Name          object 
 12  Vendor Number          float64
 13  Vendor Name            object 
 14  Item Number            object 
 15  Item Description       object 
 16  Pack                   int64  
 17  Bottle Volume (ml)     int64  
 18  State Bottle Cost      float64
 19  State Bottle Retail    float64
 20  Bottles Sold           int64  
 21  Sale (Dollars)         float64
 22  Volume Sold (Lit

In [10]:
np.sum(tr.isna())

Invoice/Item Number            0
Date                           0
Store Number                   0
Store Name                     0
Address                    78859
City                       78858
Zip Code                   78903
Store Location           1996607
County Number             155309
County                    155308
Category                   13472
Category Name              21124
Vendor Number                  0
Vendor Name                    0
Item Number                    0
Item Description               0
Pack                           0
Bottle Volume (ml)             0
State Bottle Cost             10
State Bottle Retail           10
Bottles Sold                   0
Sale (Dollars)                10
Volume Sold (Liters)           0
Volume Sold (Gallons)          0
dtype: int64

In [11]:
tr.shape

(20809628, 24)

In [12]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20809628 entries, 0 to 20965996
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Invoice/Item Number    object 
 1   Date                   object 
 2   Store Number           int64  
 3   Store Name             object 
 4   Address                object 
 5   City                   object 
 6   Zip Code               object 
 7   Store Location         object 
 8   County Number          float64
 9   County                 object 
 10  Category               float64
 11  Category Name          object 
 12  Vendor Number          float64
 13  Vendor Name            object 
 14  Item Number            object 
 15  Item Description       object 
 16  Pack                   int64  
 17  Bottle Volume (ml)     int64  
 18  State Bottle Cost      float64
 19  State Bottle Retail    float64
 20  Bottles Sold           int64  
 21  Sale (Dollars)         float64
 22  Volume Sold (Lit

#### Products
- Define a new dataframe called products, cleaning up the inconsistency.

*The same liquor item might be mapped to different categories or names at different time.* 

*The same item (using its item number as the primary key) might have different packs and inconsistent 'bottles sold' information or different item descriptions.*

*The names of the categories are time dependent, somtimes producing over-fragmentation.*

In [13]:
products = tr.filter(['Item Number', 'Item Description', 'Category', 'Category Name'])

In [14]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20809628 entries, 0 to 20965996
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Item Number       object 
 1   Item Description  object 
 2   Category          float64
 3   Category Name     object 
dtypes: float64(1), object(3)
memory usage: 793.8+ MB


In [15]:
products['Item Number'].value_counts()

11788     209881
36308     197669
35918     145510
64858     125958
11776     117097
           ...  
88522        101
905443       101
922756       101
77225        100
2313         100
Name: Item Number, Length: 4008, dtype: int64

In [16]:
products['Item Description'].value_counts()

Black Velvet                      541447
Hawkeye Vodka                     473170
Fireball Cinnamon Whiskey         259274
Five O'clock Vodka                257207
Jim Beam                          221947
                                   ...  
UV Salty Watermelon Mini DISCO         1
Stoli Crushed Pineapple Mini           1
The Glenlivet W/2 Glasses              1
Millet Armagnac VSOP                   1
Capel Premium Pisco DISCO              1
Name: Item Description, Length: 3946, dtype: int64

In [17]:
products['Category'].value_counts()

1012100.0    2022159
1031100.0    1742396
1031080.0    1264747
1011200.0    1246116
1031200.0    1040548
              ...   
1901100.0         62
1701200.0         43
1091000.0         40
1092000.0         33
1011800.0         28
Name: Category, Length: 99, dtype: int64

In [18]:
products['Category Name'].value_counts()

American Vodkas                         1643427
VODKA 80 PROOF                          1264747
Canadian Whiskies                       1087377
CANADIAN WHISKIES                        934782
Straight Bourbon Whiskies                705123
                                         ...   
Delisted Items                               62
Holiday VAP                                  43
American Distilled Spirits Specialty         40
Imported Distilled Spirits Specialty         33
Iowa Distillery Whiskies                     28
Name: Category Name, Length: 127, dtype: int64

In [19]:
products['Category'] = products['Category'].fillna('None')
products['Category Name'] = products['Category Name'].fillna('N/A')

For Items that have multiple different Item Descriptions, replace Item Description with the mode value so Items and Item Descriptions match 1:1.

In [20]:
item_df = products.groupby('Item Number')['Item Description'].agg(pd.Series.mode).to_frame()
item_df.reset_index(inplace=True)
item_df = item_df.rename(columns={'index':'Item Number', 'Item Description': 'Item Description (cleaned)'})
item_df

,Item Number,Item Description (cleaned)
0,115,Crown Royal w/Glass & Coaster
1,159,Jack Daniels Tennessee Fire w/2 Glasses
2,163,Hennessy VS Flask
3,183,Smirnoff 1.75L Glass w/2-50mls
4,208,Malibu 50ml Counter Unit
...,...,...
4003,88296,Patron Tequila Silver
4004,89191,Jose Cuervo Especial Reposado Tequila Mini
4005,89193,Jose Cuervo Especial Reposado Tequila
4006,89196,Jose Cuervo Especial Reposado Tequila


In [21]:
products = pd.merge(products, item_df, on='Item Number', how='left')

For Categories that have multiple different Category Names, replace Category Name with the mode value so Categories and Category Names match 1:1.

In [22]:
cat_df = products.groupby('Category')['Category Name'].agg(pd.Series.mode).to_frame()
cat_df.reset_index(inplace=True)
cat_df = cat_df.rename(columns={'index':'Category', 'Category Name': 'Category Name (cleaned)'})
cat_df

,Category,Category Name (cleaned)
0,1.0111e+06,Blended Whiskies
1,1.0112e+06,Straight Bourbon Whiskies
2,1.01125e+06,SINGLE BARREL BOURBON WHISKIES
3,1.0113e+06,TENNESSEE WHISKIES
4,1.0114e+06,Tennessee Whiskies
...,...,...
95,1.7011e+06,Temporary & Specialty Packages
96,1.7012e+06,Holiday VAP
97,1.9011e+06,Delisted Items
98,1.9012e+06,Special Order Items


In [23]:
products = pd.merge(products, cat_df, on='Category', how='left')

In [24]:
products = products.drop(columns=['Item Description', 'Category Name'])

In [25]:
products

,Item Number,Category,Item Description (cleaned),Category Name (cleaned)
0,80457,1.0812e+06,Ryan's Cream Liqueur,CREAM LIQUEURS
1,10008,1.0122e+06,Scoresby Rare Scotch,SCOTCH WHISKIES
2,15628,1.0124e+06,Jameson,Irish Whiskies
3,10009,1.0122e+06,Scoresby Rare Scotch,SCOTCH WHISKIES
4,100107,1.7011e+06,Captain Morgan OSR Glass w/50ml Apple Smash & ...,Temporary & Specialty Packages
...,...,...,...,...
20809623,57181,1.0711e+06,Chi-Chi's Pineapple Margarita,Cocktails /RTD
20809624,25607,1.0111e+06,Seagrams 7 Crown Bl Whiskey,Blended Whiskies
20809625,69634,1.0822e+06,Dr. Mcgillicuddy's Cherry Schnapps,Imported Schnapps
20809626,18352,1.0112e+06,Four Roses Yellow Label,Straight Bourbon Whiskies


In [26]:
pd.options.display.max_rows = 100
products['Category Name (cleaned)'].value_counts()

Canadian Whiskies                       2022159
American Vodkas                         1742396
VODKA 80 PROOF                          1264747
Straight Bourbon Whiskies               1246116
American Flavored Vodka                 1040548
Blended Whiskies                         907067
Whiskey Liqueur                          873300
TEQUILA                                  669816
PUERTO RICO & VIRGIN ISLANDS RUM         659940
Cocktails /RTD                           588632
Spiced Rum                               538860
SPICED RUM                               529549
AMERICAN DRY GINS                        478418
IMPORTED VODKA - MISC                    472046
CREAM LIQUEURS                           459353
American Schnapps                        407255
IMPORTED VODKA                           402161
American Brandies                        394218
Imported Vodkas                          380804
American Cordials & Liqueur              377111
SCOTCH WHISKIES                         

In [27]:
products['Category Name (cleaned)'] = products['Category Name (cleaned)'].map(lambda x: 
                                                                              np.where(x in ["Imported Whiskies", "Canadian Whiskies", "Irish Whiskies", "SCOTCH WHISKIES", "SINGLE MALT SCOTCH", "Single Malt Scotch"], "Imported Whiskey", 
                                                                              np.where(x in ["American Whiskies", "Iowa Distilleries", "Iowa Distillery Whiskies", "Corn Whiskies", "SINGLE BARREL BOURBON WHISKIES", "Straight Rye Whiskies", "Straight Bourbon Whiskies", "TENNESSEE WHISKIES", "Tennessee Whiskies", "Iowa Distillery Whiskies", "Blended Whiskies", "STRAIGHT RYE WHISKIES"], "American Whiskey",
                                                                              np.where(x in ["American Vodkas", "LOW PROOF VODKA", "OTHER PROOF VODKA", "American Flavored Vodka", "American Vodka", "VODKA 80 PROOF"], "American Vodka",
                                                                              np.where(x in ["Imported Vodkas", "IMPORTED VODKA", "Imported Vodka", "IMPORTED VODKA - MISC"], "Imported Vodka",
                                                                              np.where(x in ["MISCELLANEOUS  BRANDIES", "BLACKBERRY BRANDIES", "CHERRY BRANDIES", "AMERICAN GRAPE BRANDIES", "PEACH BRANDIES"], "American Brandy",
                                                                              np.where(x in ["Imported Brandies", "IMPORTED GRAPE BRANDIES"], "Imported Brandy",
                                                                              np.where(x in ["American Brandies"], "American Brandy",         
                                                                              np.where(x in ["Imported Dry Gins"], "Imported Gin",         
                                                                              np.where(x in ["American Sloe Gins", "AMERICAN DRY GINS", "Flavored Gin", "FLAVORED GINS"], "American Gin",
                                                                              np.where(x in ["Imported Rums", "PUERTO RICO & VIRGIN ISLANDS RUM", "BARBADOS RUM"], "Imported Rum",
                                                                              np.where(x in ["American Rums", "SPICED RUM", "Spiced Rum", "FLAVORED RUM", "Flavored Rum", "Gold Rum"], "American Rum", 
                                                                              np.where(x in ["Imported Cordials & Liqueurs", "ANISETTE", "IMPORTED AMARETTO", "AMARETTO - IMPORTED", "MISC. IMPORTED CORDIALS & LIQUEURS", "Imported Cordials & Liqueur"], "Imported Cordials & Liqueur",
                                                                              np.where(x in ["American Cordials & Liqueurs", "WHITE CREME DE MENTHE", "GREEN CREME DE MENTHE", "CREME DE ALMOND", "DARK CREME DE CACAO", "WHITE CREME DE CACAO", "Triple Sec", "DISTILLED SPIRITS SPECIALTY", "AMERICAN AMARETTO", "MISC. AMERICAN CORDIALS & LIQUEURS", "Whiskey Liqueur", "American Cordials & Liqueur", "CREAM LIQUEURS", "COFFEE LIQUEURS", "Coffee Liqueurs"], "American Cordials & Liqueur",
                                                                              np.where(x in ["Mezcal", "Tequilas", "TEQUILA", "100% Agave Tequila", "Cocktails /RTD", "Cocktails / RTD"], "Imported Tequila/Mezcal",
                                                                              np.where(x in ["IMPORTED SCHNAPPS"], "Imported Schnapps",  
                                                                              np.where(x in ["SPEARMINT SCHNAPPS", "MISCELLANEOUS SCHNAPPS", "TROPICAL FRUIT SCHNAPPS", "STRAWBERRY SCHNAPPS", "ROOT BEER SCHNAPPS", "GRAPE SCHNAPPS", "WATERMELON SCHNAPPS", "RASPBERRY SCHNAPPS", "APPLE SCHNAPPS", "PEACH SCHNAPPS", "CINNAMON SCHNAPPS", "BUTTERSCOTCH SCHNAPPS"], "American Schnapps",
                                                                              np.where(x in ["N/A", "American Distilled Spirit Specialty", "Imported Distilled Spirit Specialty", "AMERICAN ALCOHOL", "Neutral Grain Spirits Flavored", "Neutral Grain Spirits", "Temporary &  Specialty Packages", "Temporary & Specialty Packages", "Holiday VAP", "Special Order Items", "Delisted Items", "Distilled Spirits Specialty", "American Distilled Spirits Specialty", "Imported Distilled Spirits Specialty"], "Other", 
                                                                                       x))))))))))))))))))

In [28]:
pd.options.display.max_rows = 25
products['Category Name (cleaned)'].value_counts()

American Vodka                 4057622
American Whiskey               2881965
Imported Whiskey               2796501
American Cordials & Liqueur    2163840
American Rum                   1736773
Imported Tequila/Mezcal        1570184
Imported Vodka                 1256437
American Brandy                 728756
American Schnapps               686524
Imported Rum                    669456
Imported Cordials & Liqueur     510887
American Gin                    508709
Imported Brandy                 386616
Imported Schnapps               341282
Other                           284408
Imported Gin                    229668
Name: Category Name (cleaned), dtype: int64

In [29]:
# pd.set_option('display.max_rows', 100)
# transactions[transactions['Category Name (cleaned)']=='Neutral Grain Spirits'].groupby('Item Description (cleaned)').size()

#### Vendors

Define a new dataframe called vendors, cleaning up the inconsistency. *The same vendor (using the vendor number as its primary key) might have more than one name.*

In [30]:
vendors = tr.filter(['Vendor Number', 'Vendor Name'])

In [31]:
vendors['Vendor Number'].value_counts()

260.0    3511548
65.0     1963121
434.0    1937150
421.0    1337097
370.0    1271932
          ...   
127.0         54
662.0         48
310.0         35
477.0         19
324.0         10
Name: Vendor Number, Length: 163, dtype: int64

In [32]:
vendors['Vendor Name'].value_counts()

Jim Beam Brands         1963121
DIAGEO AMERICAS         1832104
Diageo Americas         1679444
Luxco-St Louis          1042569
Heaven Hill Brands       971554
                         ...   
3 BADGE MIXOLOGY              3
CLASSIC WINE IMPORTS          2
WINDY HILL SPIRITS            2
WERNER DISTILLING             2
WINE IS FINE INC              1
Name: Vendor Name, Length: 259, dtype: int64

In [33]:
vendors['Vendor Number'] = vendors['Vendor Number'].fillna('N/A')
vendors['Vendor Name'] = vendors['Vendor Name'].fillna('N/A')

In [34]:
vendor_df = vendors.groupby('Vendor Number')['Vendor Name'].agg(pd.Series.mode).to_frame()
vendor_df.reset_index(inplace=True)
vendor_df = vendor_df.rename(columns={'index':'Vendor Number', 'Vendor Name': 'Vendor Name (cleaned)'})
vendor_df

,Vendor Number,Vendor Name (cleaned)
0,10.0,A HARDY USA LTD
1,35.0,"Bacardi U.S.A., Inc."
2,55.0,SAZERAC NORTH AMERICA
3,65.0,Jim Beam Brands
4,79.0,BRECKENRIDGE DISTILLERY
...,...,...
158,962.0,DUGGANS DISTILLERS
159,969.0,Aveniu Brands
160,971.0,"Hood River Distillers, Inc."
161,977.0,"Werner Distilling, LLC"


In [35]:
vendors = pd.merge(vendors, vendor_df, on='Vendor Number', how='left')

In [36]:
vendors

,Vendor Number,Vendor Name,Vendor Name (cleaned)
0,330.0,Gemini Spirits,Gemini Spirits
1,421.0,SAZERAC COMPANY INC,SAZERAC COMPANY INC
2,370.0,PERNOD RICARD USA,PERNOD RICARD USA
3,421.0,SAZERAC COMPANY INC,SAZERAC COMPANY INC
4,260.0,DIAGEO AMERICAS,DIAGEO AMERICAS
...,...,...,...
20809623,55.0,SAZERAC NORTH AMERICA,SAZERAC NORTH AMERICA
20809624,260.0,DIAGEO AMERICAS,DIAGEO AMERICAS
20809625,421.0,SAZERAC COMPANY INC,SAZERAC COMPANY INC
20809626,184.0,FOUR ROSES DISTILLERY,FOUR ROSES DISTILLERY


In [37]:
vendors = vendors.drop(columns=['Vendor Name'])

In [38]:
vendors

,Vendor Number,Vendor Name (cleaned)
0,330.0,Gemini Spirits
1,421.0,SAZERAC COMPANY INC
2,370.0,PERNOD RICARD USA
3,421.0,SAZERAC COMPANY INC
4,260.0,DIAGEO AMERICAS
...,...,...
20809623,55.0,SAZERAC NORTH AMERICA
20809624,260.0,DIAGEO AMERICAS
20809625,421.0,SAZERAC COMPANY INC
20809626,184.0,FOUR ROSES DISTILLERY


#### Stores

Define a new dataframe called stores, cleaning up the inconsistency. *The same store (using its store number as the primary key) might have more than one name. The same store (using the store number as its primary key) might be associated with non-unique long-lat coordinates and non-standardized store names.*

In [39]:
stores = tr.filter(['Store Number', 'Store Name'])

In [40]:
stores['Store Number'].value_counts()

2633    173057
4829    144487
2190    135898
2512    123230
2572    108579
         ...  
9934         2
9930         2
9932         1
4669         1
9936         1
Name: Store Number, Length: 2559, dtype: int64

In [41]:
stores['Store Name'].value_counts()

Hy-Vee #3 / BDI / Des Moines           173057
Central City 2                         144487
Central City Liquor, Inc.              135898
Hy-Vee Wine and Spirits / Iowa City    123230
Hy-Vee Food Store / Cedar Falls        108579
                                        ...  
Thome Enterprises LLC                       2
Cats Eye Distillery, LLC                    2
Dash Events LLC                             1
3-Oaks Distillery, LLC                      1
Vom Fass / Des Moines                       1
Name: Store Name, Length: 2720, dtype: int64

In [42]:
store_df = stores.groupby('Store Number')['Store Name'].agg(pd.Series.mode).to_frame()
store_df.reset_index(inplace=True)
store_df = store_df.rename(columns={'index':'Store Number', 'Store Name': 'Store Name (cleaned)'})
store_df

,Store Number,Store Name (cleaned)
0,2106,Hillstreet News and Tobacco
1,2113,Jamboree Foods
2,2130,Sycamore Convenience
3,2132,Spirits and Such
4,2152,Dugans Supermarket
...,...,...
2554,9930,Thome Enterprises LLC
2555,9931,Lonely Oak Distilery
2556,9932,"3-Oaks Distillery, LLC"
2557,9934,"Cats Eye Distillery, LLC"


In [43]:
stores = pd.merge(stores, store_df, on='Store Number', how='left')

In [44]:
stores

,Store Number,Store Name,Store Name (cleaned)
0,3762,Wine and Spirits Gallery,Wine and Spirits Gallery
1,4320,Fareway Stores #470 / Perry,Fareway Stores #470 / Perry
2,2643,Hy-Vee Wine and Spirits / Waterloo,Hy-Vee Wine and Spirits / Waterloo
3,3456,Quick Shop / Clear Lake,Quick Shop / Clear Lake
4,5151,IDA Liquor,IDA Liquor
...,...,...,...
20809623,2601,Hy-Vee Food Store / Fairfield,Hy-Vee Food Store / Fairfield
20809624,2191,Keokuk Spirits,Keokuk Spirits
20809625,2605,Hy-Vee Drugstore #5 / Cedar Rapids,Hy-Vee Drugstore #5 / Cedar Rapids
20809626,4315,Fareway Stores #554 / Washington,Fareway Stores #554 / Washington


In [45]:
stores = stores.drop(columns=['Store Name'])

In [46]:
stores

,Store Number,Store Name (cleaned)
0,3762,Wine and Spirits Gallery
1,4320,Fareway Stores #470 / Perry
2,2643,Hy-Vee Wine and Spirits / Waterloo
3,3456,Quick Shop / Clear Lake
4,5151,IDA Liquor
...,...,...
20809623,2601,Hy-Vee Food Store / Fairfield
20809624,2191,Keokuk Spirits
20809625,2605,Hy-Vee Drugstore #5 / Cedar Rapids
20809626,4315,Fareway Stores #554 / Washington


In [47]:
pd.set_option('display.max_rows', 100)
stores['Store Name (cleaned)'].value_counts()[stores['Store Name (cleaned)'].value_counts() > 10000]

Hy-Vee #3 / BDI / Des Moines                    173057
Central City 2                                  144487
Central City Liquor, Inc.                       135898
Hy-Vee Wine and Spirits / Iowa City             123230
Hy-Vee Food Store / Cedar Falls                 108579
Hy-Vee Wine and Spirits / Bettendorf            107961
Hy-Vee Food Store #1 / Mason City               107625
Hy-Vee #3 Food and Drugstore                    106589
Hy-Vee #4 / WDM                                 102024
Hy-vee  #2 / Ames                               100854
Hy-Vee Food Store #1 / Ames                      99468
Hy-Vee #7 / Cedar Rapids                         98791
Hy-Vee Food and Drug / Clinton                   95519
Hy-Vee Food Store / Spencer                      93136
Hy-Vee Wine and Spirits #2                       92682
Hy-Vee Food Store #5 / Cedar Rapids              92634
Hy-Vee Food Store #2 / Council Bluffs            92498
Hy-Vee #1044 / Burlington                        91139
Hy-Vee Win

In [92]:
stores['Store Type'] = stores['Store Name (cleaned)'].map(lambda x: 
                                              np.where(any(["Osco" in x,"Fareway" in x,"FAREWAY" in x,"Hy-Vee" in x,"HY-VEE" in x,"Hy-vee" in x,"Schnucks" in x,"Shop N Save" in x]), "Chain Supermarket",
                                              np.where(any(["Quillins" in x,"Food" in x,"Mcnally's Super Valu" in x,"Saver" in x,"Big G" in x,"Grocery" in x,"Market" in x,"Mart" in x,"mart" in x,"Supermarket" in x,"Price Chopper" in x,"Dahl's" in x,"Jack" in x,"Oasis" in x]), "Small Grocery",
                                              np.where(any(["Target" in x,"Wal-Mart" in x,"Sam's Club" in x,"Costco" in x]), "Wholesale Club",         
                                              np.where(any(["Liquor" in x,"LIQUOR" in x,"Tobacco" in x,"Smoke" in x,"Bottle" in x,"Beverage" in x,"Spirits" in x,"Wine" in x,"Cork" in x,x in ["Great Pastimes","Bootleggin' Barzini's Fin","Central City 2","Benz Distributing"]]), "Liquor/Tobacco Store",
                                              np.where(any(["Stop" in x,"Station" in x,"Gas" in x,"Phillips" in x,"Convenience" in x,"Quick" in x,"Stop" in x,"Drug" in x,"Quik" in x,"QUIK" in x,"Kwik" in x,"Music Station" in x,"CVS" in x,"Walgreens" in x,"Kum & Go" in x,"KUM & GO" in x,"Store" in x,"Dyno's" in x,"DYNO'S" in x,"Pantry" in x]),"Convenience Store", x))))))

In [93]:
stores['Store Type'].value_counts()[stores['Store Type'].value_counts()>8000]

Chain Supermarket                       8163978
Liquor/Tobacco Store                    4592464
Small Grocery                           3827819
Convenience Store                       2804350
Wholesale Club                           443449
FRANKLIN STREET FLORAL & GIFT             19864
GD Xpress / Davenport                     17642
Thriftway                                 16052
New Star / Fort Dodge                     15749
Ding's Honk'n Holler                      15669
Swils                                     15216
Jim and Charlies Affiliated               14962
Sauce                                     13764
Bani's                                    13663
Montezuma Super Valu                      12496
The Secret Cellar                         12003
After 5 Somewhere                         11916
Ameristar Casino / Council Bluffs         11728
Barmuda Distribution                      11720
Ding's Honk And Holler                    11591
CENTER POINT FOODS                      

#### Prices

- Define a new dataframe called product prices, recording the historical bottle price changes of the products. 

In [50]:
tr['Date'] = pd.to_datetime(tr['Date'])

In [51]:
prices = tr.filter(['Date', 'State Bottle Retail', 'Bottles Sold', 'Sale (Dollars)', 'Volume Sold (Gallons)'])

In [90]:
prices

,Date,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Gallons)
0,2016-03-16,10.85,1,10.85,0.26
1,2019-07-15,15.75,6,94.50,2.77
2,2020-07-01,48.72,2,97.44,0.92
3,2019-07-11,10.11,2,20.22,0.52
4,2019-07-23,27.00,6,162.00,2.93
...,...,...,...,...,...
20965992,2021-03-31,9.75,30,247.50,13.86
20965993,2021-03-31,12.00,12,144.00,3.17
20965994,2021-03-31,8.00,6,48.00,0.59
20965995,2021-03-31,18.75,12,213.12,2.37


#### Transactions

Define a master dataframe called transactions, which records the vendor number, store number, product (item number), transaction date, bottles sold, volume (either in gallons or in liters), sales amount.

In [94]:
tr = reduce(lambda x,y: pd.merge(x, y, left_index=True, right_index=True), [products, vendors, stores, prices])

In [95]:
tr

,Item Number,Category,Item Description (cleaned),Category Name (cleaned),Vendor Number,Vendor Name (cleaned),Store Number,Store Name (cleaned),Store Type,Date,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Gallons)
0,80457,1.0812e+06,Ryan's Cream Liqueur,American Cordials & Liqueur,330.0,Gemini Spirits,3762,Wine and Spirits Gallery,Liquor/Tobacco Store,2016-03-16,10.85,1,10.85,0.26
1,10008,1.0122e+06,Scoresby Rare Scotch,Imported Whiskey,421.0,SAZERAC COMPANY INC,4320,Fareway Stores #470 / Perry,Chain Supermarket,2019-07-15,15.75,6,94.50,2.77
2,15628,1.0124e+06,Jameson,Imported Whiskey,370.0,PERNOD RICARD USA,2643,Hy-Vee Wine and Spirits / Waterloo,Chain Supermarket,2020-07-01,48.72,2,97.44,0.92
3,10009,1.0122e+06,Scoresby Rare Scotch,Imported Whiskey,421.0,SAZERAC COMPANY INC,3456,Quick Shop / Clear Lake,Convenience Store,2019-07-11,10.11,2,20.22,0.52
4,100107,1.7011e+06,Captain Morgan OSR Glass w/50ml Apple Smash & ...,Other,260.0,DIAGEO AMERICAS,5151,IDA Liquor,Liquor/Tobacco Store,2019-07-23,27.00,6,162.00,2.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20809623,57181,1.0711e+06,Chi-Chi's Pineapple Margarita,Imported Tequila/Mezcal,55.0,SAZERAC NORTH AMERICA,2601,Hy-Vee Food Store / Fairfield,Chain Supermarket,2021-03-09,7.74,10,77.40,0.13
20809624,25607,1.0111e+06,Seagrams 7 Crown Bl Whiskey,American Whiskey,260.0,DIAGEO AMERICAS,2191,Keokuk Spirits,Liquor/Tobacco Store,2021-03-09,44.22,2,88.44,0.52
20809625,69634,1.0822e+06,Dr. Mcgillicuddy's Cherry Schnapps,Imported Schnapps,421.0,SAZERAC COMPANY INC,2605,Hy-Vee Drugstore #5 / Cedar Rapids,Chain Supermarket,2021-03-09,15.50,3,46.50,0.59
20809626,18352,1.0112e+06,Four Roses Yellow Label,American Whiskey,184.0,FOUR ROSES DISTILLERY,4315,Fareway Stores #554 / Washington,Chain Supermarket,2021-03-10,7.26,1,7.26,0.01


In [ ]:
del [products, vendors, stores, prices]

In [ ]:
import gc
gc.collect()

In [96]:
tr = tr.drop(['Category', 'Vendor Number', 'Store Number', 'State Bottle Retail'], axis = 1)

In [97]:
tr['Item Description (cleaned)'] = tr['Item Description (cleaned)'].astype('string')

In [98]:
tr = tr[tr['Date'] <= '2021-01-01']

In [99]:
tr.to_csv('transactions.csv', index=False)

#### Create relational database

In [ ]:
# conn = sql.connect('trans.db')
# products.to_sql('products', conn)
# vendors.to_sql('vendors', conn)
# stores.to_sql('stores', conn)
# prices.to_sql('prices', conn)

In [ ]:
# test1 = pd.read_sql('SELECT * FROM stores WHERE `Store Number` == 3762', conn)

In [ ]:
# test1.head()

In [ ]:
# test2 = pd.read_sql('SELECT * FROM transactions WHERE `Item Number` == 80457', conn)

In [ ]:
# test2.head()

In [ ]:
# test3 = pd.read_sql('SELECT * FROM transactions WHERE `Category Name (cleaned)` == "VODKA 80 PROOF"', conn)

In [ ]:
# test3